# CATBoost Model Preparation


In [ ]:
#!conda install catboost -y

In [ ]:
#spark.stop()

In [1]:
# Setup Environment
import os
import logging
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import optuna
from catboost import CatBoostRanker, CatBoostRegressor, CatBoostClassifier, Pool
import numpy as np
import itertools
from sklearn.preprocessing import LabelEncoder
import pyspark.sql.functions as F
from pyspark.sql.functions import (col, count, row_number, abs, unix_timestamp, mean, 
                                   when, lit, min as F_min, max as F_max , upper, trim,
                                   row_number, mean as F_mean, countDistinct, last, first, when)
from src.data_preprocessing.data_prep1.data_utils import initialize_environment 
# Set global references to None
spark = None
master_results_df = None
race_df = None
df = None
training_data = None
train_df = None

In [3]:

spark, jdbc_url, jdbc_properties, parquet_dir, log_file = initialize_environment()


Spark session created successfully.


In [4]:
# This dataset has already been cleaned up in the LGB notebook and saved as a starting point
# It now just need to be converted to Panadas and run in the GBDT variant model (LGB, XGB, CatBoost)
training_data = spark.read.parquet("/home/exx/myCode/horse-racing/FoxRiverAIRacing/data/parquet/train_df")



In [5]:
training_data.count()

394322

# Switching to Pandas

In [6]:
# Convert Spark DataFrame -> Pandas DataFrame

training_data = training_data.toPandas()
# Quick info about the DataFrame
#print(df.info())
#print(df.head(5))

# Set race_id

In [7]:
# Create race_id for grouping
training_data["race_id"] = (
    training_data["course_cd"].astype(str) + "_" +
    training_data["race_date"].astype(str) + "_" +
    training_data["race_number"].astype(str)
)

# Group and sort data by race_id

In [8]:
group_array = training_data.groupby("race_id").size().values  # array of group sizes
print(group_array)

[ 6  8  7 ...  7 10  8]


In [9]:
# Sort by race_id for consistency
training_data = training_data.sort_values("race_id", ascending=True)

# Set Rank/Label

In [10]:
# If the largest official_fin is 20 (some races can have 20 horses),
# then label = (21 - official_fin).
# So official_fin=1 => label=20, official_fin=2 =>19, etc.
# If your max is 14, you can do (15 - official_fin).  Just ensure "best" horse has largest label.
# Create rank column and drop official_finish
if "official_fin" in training_data.columns:
    training_data["rank"] = 21 - training_data["official_fin"]
    training_data.drop(columns=["official_fin"], inplace=True)

# Simple Target Encoding

In [11]:
# Omit this step for Cat Boost.

# Set the Category Columns with Label Encoder

In [12]:
# Keep original horse_id for identification
training_data["horse_id_original"] = training_data["horse_id"]

# Encode categorical columns
cat_cols = [
    "horse_id", "course_cd", "sex", "equip", "surface", "med",  
    "race_type", "stk_clm_md", "turf_mud_mark", "layoff_cat"
]
for c in cat_cols:
    lbl = LabelEncoder()
    training_data[c] = lbl.fit_transform(training_data[c].astype(str))


# Convert DataTime columns to Numerical Values

In [13]:
# Convert datetime columns to numerical
training_data["first_race_date_5"] = pd.to_datetime(training_data["first_race_date_5"])
training_data["most_recent_race_5"] = pd.to_datetime(training_data["most_recent_race_5"])
training_data["prev_race_date"] = pd.to_datetime(training_data["prev_race_date"])

# Calculate numeric date features
training_data["first_race_date_5_numeric"] = (training_data["first_race_date_5"] - pd.Timestamp("1970-01-01")).dt.days
training_data["most_recent_race_5_numeric"] = (training_data["most_recent_race_5"] - pd.Timestamp("1970-01-01")).dt.days
training_data["prev_race_date"] = (training_data["prev_race_date"] - pd.Timestamp("1970-01-01")).dt.days

# Drop original datetime columns
training_data.drop(columns=["first_race_date_5", "most_recent_race_5", "prev_race_date"], inplace=True)


# Assigned Numerical Features

In [14]:
# Define features and target
features = [
    'horse_id', 'course_cd', 'sex', 'equip', 'surface', 'med',
    'race_type', 'stk_clm_md', 'turf_mud_mark', 'layoff_cat',
    'race_number', 'purse', 'weight', 'claimprice', 'power', 'morn_odds',
    'avgspd', 'class_rating', 'net_sentiment', 'avg_spd_sd', 'ave_cl_sd',
    'hi_spd_sd', 'pstyerl', 'all_starts', 'all_win', 'all_place',
    'all_show', 'all_fourth', 'all_earnings', 'cond_starts', 'cond_win',
    'cond_place', 'cond_show', 'cond_fourth', 'cond_earnings',
    'avg_speed_5', 'best_speed', 'avg_beaten_len_5', 'avg_dist_bk_gate1_5',
    'avg_dist_bk_gate2_5', 'avg_dist_bk_gate3_5', 'avg_dist_bk_gate4_5',
    'avg_speed_fullrace_5', 'avg_stride_length_5', 'avg_strfreq_q1_5',
    'avg_strfreq_q2_5', 'avg_strfreq_q3_5', 'avg_strfreq_q4_5',
    'prev_speed', 'speed_improvement', 'days_off', 'avg_workout_rank_3',
    'jock_win_percent', 'jock_itm_percent', 'trainer_win_percent',
    'trainer_itm_percent', 'jt_win_percent', 'jt_itm_percent',
    'jock_win_track', 'jock_itm_track', 'trainer_win_track',
    'trainer_itm_track', 'jt_win_track', 'jt_itm_track', 'age_at_race_day',
    'distance_meters', 'count_workouts_3'
]
X_all = training_data[features]  # Keep as DataFrame
y_all = training_data["rank"]    # Target variable
race_ids = training_data["race_id"]  # For grouping


# Split Data Into Training and Validation

In [15]:
# Split train/valid with race-based grouping
def split_train_valid(training_data, race_id_col, features, target, train_ratio=0.8, random_state=42):
    unique_races = training_data[race_id_col].unique()
    unique_races = shuffle(unique_races, random_state=random_state)

    cut = int(len(unique_races) * train_ratio)
    train_races = set(unique_races[:cut])
    valid_races = set(unique_races[cut:])

    train_mask = training_data[race_id_col].isin(train_races)
    valid_mask = training_data[race_id_col].isin(valid_races)

    X_train = training_data.loc[train_mask, features]
    y_train = training_data.loc[train_mask, target]
    race_id_train = training_data.loc[train_mask, race_id_col].values

    X_valid = training_data.loc[valid_mask, features]
    y_valid = training_data.loc[valid_mask, target]
    race_id_valid = training_data.loc[valid_mask, race_id_col].values

    return X_train, y_train, race_id_train, X_valid, y_valid, race_id_valid


In [16]:

X_train, y_train, race_id_train, X_valid, y_valid, race_id_valid = split_train_valid(
    training_data=training_data,
    race_id_col="race_id",
    features=features,
    target="rank",
    train_ratio=0.8
)


In [17]:
#print (X_train)

# Make Group Array

In [25]:
# Create group arrays with numeric group IDs
def make_group_array(race_id_array):
    """
    Convert race_id strings to unique numeric IDs and create group_id array.
    """
    race_id_numeric, _ = pd.factorize(race_id_array)
    return race_id_numeric

group_train = make_group_array(race_id_train)
group_valid = make_group_array(race_id_valid)

# Validate lengths
assert len(group_train) == len(X_train), "Mismatch between group_train and X_train"
assert len(group_valid) == len(X_valid), "Mismatch between group_valid and X_valid"

In [26]:
# Validate groups
print("X_train shape:", X_train.shape)
print("group_train length:", len(group_train))

X_train shape: (315386, 67)
group_train length: 315386


# CATBoost

# Run using Optuna or Hyperparameter Tuning

In [39]:
print("X_valid shape:", X_valid.shape)
print("y_valid shape:", y_valid.shape)
print("group_valid length:", len(group_valid))

assert len(X_valid) == len(y_valid), "Mismatch between X_valid and y_valid"
assert len(X_valid) == len(group_valid), "Mismatch between X_valid and group_valid"

X_valid shape: (78936, 67)
y_valid shape: (78936,)
group_valid length: 78936


In [40]:
print("Evaluation results:", model.get_evals_result())

NameError: name 'model' is not defined

In [57]:
import sys
import optuna
from catboost import Pool, CatBoostRanker
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import ndcg_score
import pandas as pd
import numpy as np

# Define the function to calculate NDCG
def calculate_ndcg(y_true, y_pred, group):
    """
    Calculate NDCG score for the given true and predicted values.
    
    :param y_true: True labels
    :param y_pred: Predicted scores
    :param group: Group information for ranking
    :return: NDCG score
    """
    # Split the data into groups
    start_idx = 0
    ndcg_scores = []
    for group_size in group:
        end_idx = start_idx + group_size
        y_true_group = y_true[start_idx:end_idx]
        y_pred_group = y_pred[start_idx:end_idx]
        if len(y_true_group) > 0 and len(y_pred_group) > 0:
            ndcg_scores.append(ndcg_score([y_true_group], [y_pred_group], k=4))
        start_idx = end_idx
    
    # Return the average NDCG score
    return np.mean(ndcg_scores)

# Define the objective function for Optuna
def objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 100, 2000),
        "depth": trial.suggest_int("depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1.0, 10.0),
        "random_seed": 42,
        "loss_function": "YetiRank",
        "eval_metric": "NDCG:top=4",
        "task_type": "GPU",
        "devices": "0:1"  # Adjust for multiple GPUs
    }

    train_pool = Pool(data=X_train, label=y_train, group_id=group_train, cat_features=cat_cols)
    valid_pool = Pool(data=X_valid, label=y_valid, group_id=group_valid, cat_features=cat_cols)

    model = CatBoostRanker(**params)
    model.fit(train_pool, eval_set=valid_pool, verbose=False, early_stopping_rounds=50)

    # Evaluate on CPU
    valid_predictions = model.predict(valid_pool)
    ndcg_score_value = calculate_ndcg(y_valid, valid_predictions, group_valid)

    return -ndcg_score_value

# Define the function to run Optuna
def run_optuna():
    study = optuna.create_study(direction="minimize")  # Inverted NDCG
    study.optimize(objective, n_trials=50)
    return study

# Figure out which modes to run
args = sys.argv[1:]
run_prediction = False
run_train = False
cleanse = False
training_data = None
predict = None
if not args:
    # No arguments => run both
    run_prediction = True
    run_train = True
    cleanse = False
else:
    # Parse each argument
    for arg in args:
        arg_lower = arg.lower()
        if arg_lower == "prediction":
            run_prediction = True
        elif arg_lower == "train":
            run_train = True
        elif arg_lower == "cleanse":
            cleanse = True
        else:
            print(f"Unknown argument: {arg}. Valid arguments are 'prediction', 'train', or 'cleanse'.")
            sys.exit(1)

spark = None
try:
    # Read configuration
    # ... (your existing code to read configuration)

    if run_train:
        # Load and prepare training data
        # ... (your existing code to load and prepare training data)

        # Ensure group sizes are correct
        group_train = X_train.groupby('race_id').size().to_numpy()
        group_valid = X_valid.groupby('race_id').size().to_numpy()

        # Run Optuna optimization
        study = run_optuna()
        best_params = study.best_params
        best_value = -study.best_value  # Undo inversion to get positive NDCG@4

        print("Best Params:", best_params)
        print("Best NDCG@4:", best_value)

    if run_prediction:
        pass
        # Load and prepare prediction data
        # ... (your existing code to load and prepare prediction data)

    if cleanse:
        pass
        # Perform data cleansing
        # ... (your existing code to perform data cleansing)

except Exception as e:
    print(f"An error occurred: {e}")
    sys.exit(1)
finally:
    if spark:
        spark.stop()

Unknown argument: -f. Valid arguments are 'prediction', 'train', or 'cleanse'.


SystemExit: 1

/home/exx/anaconda3/envs/mamba_env/envs/rapids-23.08/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [76]:
from catboost import Pool, CatBoostRanker
import optuna
import numpy as np

# Define the Optuna objective function
def objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 100, 2000),
        "depth": trial.suggest_int("depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1.0, 10.0),
        "random_seed": 42,
        "loss_function": "YetiRank",
        "eval_metric": "NDCG:top=4",
        "task_type": "GPU",
        "devices": "0:1",
    }

    train_pool = Pool(data=X_train, label=y_train, group_id=group_train, cat_features=cat_cols)
    valid_pool = Pool(data=X_valid, label=y_valid, group_id=group_valid, cat_features=cat_cols)

    # Train the model
    model = CatBoostRanker(**params)
    model.fit(
        train_pool,
        eval_set=[valid_pool],
        verbose=50,  # Log progress every 50 iterations
        early_stopping_rounds=50
    )

    # Extract and summarize validation results
    evals_result = model.get_evals_result()
    validation_scores = evals_result["validation"]["NDCG:top=4;type=Base"]
    print(f"First validation NDCG@4: {validation_scores[0]}")
    print(f"Last validation NDCG@4: {validation_scores[-1]}")
    print(f"Best validation NDCG@4: {max(validation_scores)}")

    # Return the best score for Optuna
    return -max(validation_scores)  # Negate for Optuna minimization

# Run Optuna
def run_optuna():
    study = optuna.create_study(direction="minimize")  # Inverted NDCG
    study.optimize(objective, n_trials=50)  # Run 50 trials
    return study

study = run_optuna()

# Print the best parameters and NDCG score
best_params = study.best_params
best_value = -study.best_value  # Undo inversion to get positive NDCG
print("Best Params:", best_params)
print("Best NDCG@4:", best_value)

[I 2025-01-11 16:13:18,127] A new study created in memory with name: no-name-7ab6a95a-8aa2-4afd-a381-da991587c3e9


Groupwise loss function. OneHotMaxSize set to 10
0:	test: 0.9341278	best: 0.9341278 (0)	total: 12ms	remaining: 20.3s


Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=4;type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


50:	test: 0.9475019	best: 0.9479235 (35)	total: 561ms	remaining: 18s


[I 2025-01-11 16:13:20,003] Trial 0 finished with value: -0.9479235406633758 and parameters: {'iterations': 1691, 'depth': 5, 'learning_rate': 0.001509650456580058, 'l2_leaf_reg': 6.314765264745556}. Best is trial 0 with value: -0.9479235406633758.


...

1612:	test: 0.9826379	best: 0.9826379 (1612)	total: 35s	remaining: 0us
bestTest = 0.9826378837
bestIteration = 1612
First validation NDCG@4: 0.9495466339786791
Last validation NDCG@4: 0.9826378837154743
Best validation NDCG@4: 0.9826378837154743
Groupwise loss function. OneHotMaxSize set to 10
0:	test: 0.9515775	best: 0.9515775 (0)	total: 25.5ms	remaining: 30.4s


Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=4;type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


50:	test: 0.9699985	best: 0.9699985 (50)	total: 1.28s	remaining: 28.6s
100:	test: 0.9738668	best: 0.9738668 (100)	total: 2.61s	remaining: 28.1s
150:	test: 0.9761248	best: 0.9761248 (150)	total: 3.96s	remaining: 27.2s
200:	test: 0.9774454	best: 0.9774454 (200)	total: 5.31s	remaining: 26.2s
250:	test: 0.9785765	best: 0.9785765 (250)	total: 6.67s	remaining: 25s
300:	test: 0.9792480	best: 0.9792480 (300)	total: 8.04s	remaining: 23.8s
350:	test: 0.9798188	best: 0.9798188 (350)	total: 9.41s	remaining: 22.5s
400:	test: 0.9802564	best: 0.9802564 (400)	total: 10.8s	remaining: 21.2s
450:	test: 0.9805970	best: 0.9805970 (450)	total: 12.1s	remaining: 19.9s
500:	test: 0.9808744	best: 0.9808786 (499)	total: 13.5s	remaining: 18.6s
550:	test: 0.9812160	best: 0.9812160 (550)	total: 14.9s	remaining: 17.3s
600:	test: 0.9814075	best: 0.9814223 (598)	total: 16.2s	remaining: 15.9s
650:	test: 0.9816449	best: 0.9816449 (650)	total: 17.6s	remaining: 14.6s
700:	test: 0.9818805	best: 0.9818805 (700)	total: 19s	r

[I 2025-01-11 16:34:52,164] Trial 49 finished with value: -0.9831027206576761 and parameters: {'iterations': 1191, 'depth': 10, 'learning_rate': 0.22039662941170574, 'l2_leaf_reg': 9.008350201576475}. Best is trial 31 with value: -0.9838250051054743.


1190:	test: 0.9831027	best: 0.9831027 (1190)	total: 32.3s	remaining: 0us
bestTest = 0.9831027207
bestIteration = 1190
First validation NDCG@4: 0.951577531290094
Last validation NDCG@4: 0.9831027206576761
Best validation NDCG@4: 0.9831027206576761
Best Params: {'iterations': 1368, 'depth': 10, 'learning_rate': 0.2963992234657153, 'l2_leaf_reg': 7.944911686368604}
Best NDCG@4: 0.9838250051054743


# Re-train on Final Best Model

In [87]:
# Ensure eval_metric is set
best_params["eval_metric"] = "NDCG:top=4"

# Train the model with evaluation set
final_model = CatBoostRanker(**best_params)
final_model.fit(
    full_pool,
    eval_set=[valid_pool],  # Pass validation set
    verbose=50,
    early_stopping_rounds=50
)

# Retrieve and print metrics
final_scores = final_model.get_best_score()
print("Final Training Metrics:", final_scores.get("learn", {}))
print("Final Validation Metrics:", final_scores.get("validation", {}))

# Debug evaluation results
evals_result = final_model.get_evals_result()
print("Evaluation Results:", evals_result)

Groupwise loss function. OneHotMaxSize set to 10
0:	test: 0.9502211	best: 0.9502211 (0)	total: 328ms	remaining: 7m 27s
50:	test: 0.9795611	best: 0.9795611 (50)	total: 13.8s	remaining: 5m 56s
100:	test: 0.9819861	best: 0.9819861 (100)	total: 26.7s	remaining: 5m 34s
150:	test: 0.9834282	best: 0.9834282 (150)	total: 39.5s	remaining: 5m 18s
200:	test: 0.9843255	best: 0.9843255 (200)	total: 51.6s	remaining: 4m 59s
250:	test: 0.9847605	best: 0.9847610 (249)	total: 1m 3s	remaining: 4m 44s
300:	test: 0.9850316	best: 0.9851360 (283)	total: 1m 16s	remaining: 4m 30s
350:	test: 0.9856826	best: 0.9856889 (349)	total: 1m 28s	remaining: 4m 17s
400:	test: 0.9859771	best: 0.9859771 (400)	total: 1m 41s	remaining: 4m 4s
450:	test: 0.9864049	best: 0.9864049 (450)	total: 1m 53s	remaining: 3m 51s
500:	test: 0.9864078	best: 0.9866284 (480)	total: 2m 6s	remaining: 3m 38s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9866284213
bestIteration = 480

Shrink model to first 481 iterations.
Fi

In [88]:
print(f"Validation Pool Size: {len(valid_pool.get_label())}")

Validation Pool Size: 78936


In [83]:
print("Unique group IDs:", len(np.unique(group_full)))
print("Sample group IDs:", np.unique(group_full)[:10])

Unique group IDs: 40860
Sample group IDs: [0 1 2 3 4 5 6 7 8 9]


In [89]:
# Save the final model
final_model.save_model("/home/exx/myCode/horse-racing/FoxRiverAIRacing/src/models/catboost_983825_2025-01-11_final.cbm")

In [90]:
metrics_result = final_model.eval_metrics(
    data=full_pool,
    metrics=['NDCG:top=4;type=Base'],  
    eval_period=1
)

# metrics_result is a dict: 
# {
#   "NDCG:top=5": [ value_after_tree_1, value_after_tree_2, ..., value_after_tree_N ]
# }
# So if you trained 500 trees, you'll get 500 values in the list.

last_ndcg = metrics_result['NDCG:top=5;type=Base'][-1]
print("Final NDCG@5 on full_pool:", last_ndcg)

KeyError: 'NDCG:top=5;type=Base'